In [3]:
#prepare input and output directories
import os
import re
import time
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from skimage import data, color, filters,measure,draw
import matplotlib.image as mpimg
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import traceback
from matplotlib.patches import Circle
import cv2

# Define el número máximo de hilos a utilizar (desactivado por no ser necesario para evitar problemas de memoria)
MAX_THREADS = 12

#default pixel size
DEFAULT_PIXED_SIZE = None

SIZE_REDUCTION_RESOLUTION = 4

ONLY_ONE = False

SAMPLE_NUM = 0


def processImages(inputDir,outputDir,measure_distance,get_measure=True,calcCircles=True,useDefaultScale=False,scale_path=None,join_data=False,onlyOne=False,sampleNum=None):
    global ONLY_ONE
    global SAMPLE_NUM
    ONLY_ONE = onlyOne
    SAMPLE_NUM = sampleNum

    #create output directory
    if not os.path.exists(outputDir):
        os.makedirs(outputDir)
    if useDefaultScale:
        if not os.path.exists(scale_path):
            print("Scale path does not exist")
            return
        else:
            #gets the directory where the scale is located
            scaleDir = os.path.dirname(scale_path)
            relativeDir = os.path.relpath(inputDir, scaleDir)
            newOutputDir = os.path.join(outputDir, relativeDir)
            detectMeasure(scale_path, newOutputDir, outputDir,setDefaultScale=True)
    #create results csv (for the moment it is empty)
    with open(os.path.join(outputDir, 'globalResults.csv'), 'w') as f:
        pass
    navigateDir(inputDir, outputDir,get_measure,calcCircles,measure_distance,join_data)


#navigate through the input directory
def navigateDir(inputDir,outputDir,get_measure,calcCircles,measure_distance,join_data):
    # navigates through the input directory until if gets to a directory with the name "sample X"
    for root, dirs, files in os.walk(inputDir):
        for dir in dirs:
            if (re.match(r'sample \d+', os.path.basename(dir)) or re.match(r'Sample \d+', os.path.basename(dir))):
                print("Processing directory: " + dir)
                processDir(os.path.join(root, dir),inputDir, outputDir,calcCircles,measure_distance,join_data)

def detect_measure_contours(img,xdim=-250,ydim=-500,contour_level=0.8):
    #a escala de grises y se toma solo una parte mínima
    gray_image = color.rgb2gray(img)

    # Otsu's thresholding
    thresh = filters.threshold_otsu(gray_image)
    binary_inv = gray_image > thresh

    # Blur img
    blur = filters.gaussian(binary_inv, sigma=1)

    # Otsu's thresholding
    thresh = filters.threshold_otsu(blur)
    binary = blur > thresh

    # Find contours
    contours = measure.find_contours(binary, level=contour_level)

    return contours,img

def draw_contours_in_empty(image, contours):
    # plot contours
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4))
    ax.imshow(image, cmap='gray')
    for contour in contours:
        ax.plot(contour[:, 1], contour[:, 0], linewidth=2)
    ax.axis('image')
    ax.set_axis_off()
    ax.set_title('Contours')

    fig.canvas.draw()

    data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    plt.close(fig)

    return data

def draw_main_rectangle_in_image_and_get_main_contour(image, contours):
    mask = np.ones(image.shape, dtype="uint8")
    definitive_contour = None
    #dibuja un rectangulo en la imagen para cada contorno
    for contour in contours:
        #minr, minc, maxr, maxc = measure.regionprops(contour.astype(int))[0].bbox
        minr = np.min(contour[:, 0])  # minimo valor de y
        minc = np.min(contour[:, 1])  # minimo valor de x
        maxr = np.max(contour[:, 0])  # maximo valor de y
        maxc = np.max(contour[:, 1])  # maximo valor de x
        print( "minr: ", minr, " minc: ", minc, " maxr: ", maxr, " maxc: ", maxc)
        if (maxr - minr) * (maxc - minc) > 1000:
            if definitive_contour is None or (maxr - minr) * (maxc - minc) > (np.max(definitive_contour[:, 0]) - np.min(definitive_contour[:, 0])) * (np.max(definitive_contour[:, 1]) - np.min(definitive_contour[:, 1])):
                definitive_contour = contour

    if definitive_contour is not None:
        minr = np.min(definitive_contour[:, 0])  # minimo valor de y
        minc = np.min(definitive_contour[:, 1])  # minimo valor de x
        maxr = np.max(definitive_contour[:, 0])  # maximo valor de y
        maxc = np.max(definitive_contour[:, 1])  # maximo valor de x
        rr, cc = draw.rectangle_perimeter(start=(minr, minc), extent=(maxr-minr, maxc-minc), shape=image.shape)
        mask[rr, cc] = 0
        for r, c in zip(rr, cc):
            image[r, c] = [0, 255, 0]

        #res_final = imgNew * mask[:, :]
    return definitive_contour, image

def calculate_pixel_size(definitive_contour,distance):
    minr = np.min(definitive_contour[:, 0])  # minimo valor de y
    minc = np.min(definitive_contour[:, 1])  # minimo valor de x
    maxr = np.max(definitive_contour[:, 0])  # maximo valor de y
    maxc = np.max(definitive_contour[:, 1])  # maximo valor de x

    #calculate distancia de un pixel en metros sabiendo que la distancia en horizontal del rectangulo es la siguiente
    distance = 1000
    distance_pixels = maxc - minc
    print('each pixel is ', distance/distance_pixels, ' micrometers')
    return distance/distance_pixels

def detectMeasure(imagePath, newOutputDir, outputDir,setDefaultScale=False,defaultPixelSize=None):
    pixel_size = defaultPixelSize
    if (defaultPixelSize == None):
        image = mpimg.imread(imagePath)
        # Poner a 0 todos los píxeles que no sean rojos
        red_pixels = image.copy()
        red_pixels[(red_pixels[:, :, 0] < 200) | (red_pixels[:, :, 1] >= 100) | (red_pixels[:, :, 2] >= 100)] = 0

        #tamaño real de la imagen
        contours,reduced_img = detect_measure_contours(red_pixels)

        image_with_distance = draw_contours_in_empty(reduced_img, contours)
        #obtiene el rectangulo y el contorno principal
        main_contour, image_with_main_rectangle = draw_main_rectangle_in_image_and_get_main_contour(reduced_img, contours)
        print("------------------------------------")
        print("Adding scale: " + imagePath)
        image_base_name, image_extension = os.path.splitext(os.path.basename(imagePath))
        if image_extension.lower() == '.tif':
            image_extension = '.tiff'
        new_image_name = image_base_name + "_with_distance" + image_extension
        plt.imsave(os.path.join(newOutputDir, new_image_name), image_with_main_rectangle)
        #calculate el tamaño del pixel
        pixel_size = calculate_pixel_size(main_contour, measure_distance)

    if setDefaultScale:
        global DEFAULT_PIXED_SIZE
        DEFAULT_PIXED_SIZE = pixel_size
        print("Default pixel size: ", DEFAULT_PIXED_SIZE)
        return

    # Lee el archivo CSV existente
    df = pd.read_csv(os.path.join(newOutputDir, 'results.csv'))

    # Añade una nueva columna
    df['realDiameter'] = df['diameter_pixel'] * pixel_size

    # Guarda el DataFrame de nuevo en el archivo CSV
    df.to_csv(os.path.join(newOutputDir, 'results.csv'), index=False)
    if imagePath:
        print("Scale added: " + imagePath)
    else:
        print("Default scale added")

def joinGlobal(newOutputDir, outputDir):
    dfNew = pd.read_csv(os.path.join(newOutputDir, 'results.csv'))
    #get newOutputDir name
    sampleName = os.path.basename(newOutputDir)
    dfNew = dfNew.assign(sample=sampleName)
    global_results_path = os.path.join(outputDir, 'globalResults.csv')
    if os.path.exists(global_results_path) and os.path.getsize(global_results_path) > 0:
        dfGlobal = pd.read_csv(global_results_path)
        dfGlobal = pd.concat([dfGlobal, dfNew], axis=0)
    else:
        dfGlobal = dfNew

    # Guarda el DataFrame de nuevo en el archivo CSV
    dfGlobal.to_csv(os.path.join(outputDir, 'globalResults.csv'), index=False)

def processDir(inputDir,originalInputDir, outputDir,calcCircles,measure_distance,join_data):
    if ((not ONLY_ONE) or (ONLY_ONE and SAMPLE_NUM == int(os.path.basename(inputDir).split()[1]))):

        #crea un csv de resultados reflejando la estructura del directorio de entrada en el directorio de salida
        #para ello toma la ruta relativa del directorio de entrada y crea la estructura del directorio de salida
        relativeDir = os.path.relpath(inputDir, originalInputDir)
        newOutputDir = os.path.join(outputDir, relativeDir)
        print(newOutputDir)
        if not os.path.exists(newOutputDir):
            os.makedirs(newOutputDir)

        if calcCircles:
            if ((not ONLY_ONE) or (ONLY_ONE and SAMPLE_NUM == int(os.path.basename(inputDir).split()[1]))):
                with open(os.path.join(newOutputDir, 'results.csv'), 'w') as f:
                    f.write("x,y,diameter_pixel,imagePath\n")
                        #pass
                    sampleName = os.path.basename(newOutputDir)
                    sampleNum = int(sampleName.split()[1])
                    # navega a través de los archivos en el directorio de entrada
                with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
                    start_time = time.time()
                    for root, dirs, files in os.walk(inputDir):
                        # crea una lista para contener los futures
                        futures = []
                        scaleFile = None
                        for file in files:
                            #comprueba si el archivo es una imagen
                            if re.match(r'.*\.(jpg|jpeg|png|gif|bmp|tif)$', file):
                                #comprueba si get_measure es verdadero y si la imagen contiene la palabra scale
                                if re.match(r'.*scale.*', file) or re.match(r'.*Scale.*', file) or re.match(r'.*escala.*', file) or re.match(r'.*Escala.*', file):
                                    print("Scale detected: " + file)
                                    scaleFile = file
                                    pass
                                elif calcCircles:
                                    #detecta los círculos en la imagen
                                    print("Processing image: " + file)
                                    #crea un hilo para cada imagen
                                    #futures.append(executor.submit(detectCircles, os.path.join(root, file), newOutputDir, outputDir,sampleNum))
                                    detectCircles(os.path.join(root, file), newOutputDir, outputDir,sampleNum)
                        #espera a que todos los hilos se completen
                        for future in as_completed(futures):
                            pass
                    if get_measure:
                        if scaleFile:
                            #detecta la medida en la imagen
                            print("Processing scale: " + scaleFile)
                            print("------------------------------------")
                            #crea un hilo para cada imagen
                            #executor.submit(detectMeasure, os.path.join(root, scaleFile), newOutputDir, outputDir)
                            detectMeasure(os.path.join(root, scaleFile), newOutputDir, outputDir)
                        else:
                            #detecta la medida en la imagen
                            print("Processing scale using default scale: ")
                            print("------------------------------------")
                            #crea un hilo para cada imagen
                            #executor.submit(detectMeasure, None, newOutputDir, outputDir,defaultPixelSize=DEFAULT_PIXED_SIZE)
                            detectMeasure(None, newOutputDir, outputDir,defaultPixelSize=DEFAULT_PIXED_SIZE)
                        end_time = time.time()

                        print(f"Time elapsed: {end_time - start_time} seconds")
            if join_data:
                joinGlobal(newOutputDir, outputDir)

# en newOutputDir guardaremos las imágenes con los círculos detectados y los resultados de la detección de la muestra específica y en outputDir guardaremos los resultados de la detección para tener todas las detecciones en el mismo archivo
def detectCircles(imagePath, newOutputDir, outputDir,sampleNum):
    try:
        print("Processing image: " + imagePath)
        # carga img
        image = mpimg.imread(imagePath)
        #reduce el tamaño de la imagen
        image = image[::SIZE_REDUCTION_RESOLUTION, ::SIZE_REDUCTION_RESOLUTION]

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT_ALT, 1.5, minDist=10, param1=300, param2=0.9, minRadius=10, maxRadius=500)

        circles = np.round(circles[0, :]).astype("int")
        cx, cy, radii = [], [], []

        for (x, y, r) in circles:
            cx.append(x)
            cy.append(y)
            radii.append(r)


        #dibuja los círculos en la imagen original
        image_with_circles = image.copy()
        fig, ax = plt.subplots()
        ax.imshow(image_with_circles)

        i = 0
        for center_y, center_x, radius in zip(cy, cx, radii):
            #print(f"Circle {i+1}: center: ({center_x}, {center_y}), radius: {radius*2*SIZE_REDUCTION_RESOLUTION}")
            i += 1
            circle = Circle((center_x, center_y), radius, fill=False, edgecolor='red')
            ax.add_patch(circle)

        #dibuja circulos adicionales para crear un grosor
        thickness= 5
        for i in range(1, thickness):
            for center_y, center_x, radius in zip(cy, cx, radii):
                circle = Circle((center_x, center_y), radius+i, fill=False, edgecolor='red')
                ax.add_patch(circle)

            #un loop aparte para poner el numero de cada circulo evitando que se superpongan con los círculos
            i = 0
            for center_y, center_x, radius in zip(cy, cx, radii):
                #print(f"Circle {i+1}: center: ({center_x}, {center_y}), radius: {radius*2*SIZE_REDUCTION_RESOLUTION}")
                i += 1
                circle = Circle((center_x, center_y), radius, fill=False, edgecolor='red')
                ax.add_patch(circle)
                # Annotate the circle number at the center of the circle
                ax.annotate(str(i+1), (center_x, center_y), color='blue', fontsize=12)

        #guarda la imagen con los círculos
        canvas = FigureCanvas(fig)
        canvas.draw()
        image_from_plot = canvas.buffer_rgba()
        image_with_circles = np.asarray(image_from_plot)



        image_base_name, image_extension = os.path.splitext(os.path.basename(imagePath))
        if image_extension.lower() == '.tif':
            image_extension = '.tiff'
        new_image_name = image_base_name + "_with_circles" + image_extension
        plt.imsave(os.path.join(newOutputDir, new_image_name), image_with_circles)


        print("************************************")
        print("Image processed: " + imagePath)

        with open(os.path.join(newOutputDir, 'results.csv'), 'a') as f:
            #añade las columnas al csv (que ya tienen el encabezado)
            for center_y, center_x, radius in zip(cy, cx, radii):
                f.write(f"{center_x},{center_y},{radius*2*SIZE_REDUCTION_RESOLUTION},{imagePath}\n")

        print("Image data stored: " + imagePath)

    except Exception as e:
        tb = traceback.format_exc()
        print(f"Error ocurrido en el hilo: {e}\n{tb}")







from tkinter import *
from tkinter import ttk, filedialog
import sv_ttk
import datetime
import os

def save_settings(input_dir, output_dir, measure_distance):
    with open('settings.txt', 'w') as f:
        f.write(f"input={input_dir}\n")
        f.write(f"output={output_dir}\n")
        f.write(f"calc_circles={calc_circles.get()}\n")
        f.write(f"get_measure={get_measure.get()}\n")
        f.write(f"measure_distance={measure_distance.get()}\n")
        f.write(f"useDefaultScale={useDefaultScale.get()}\n")
        f.write(f"scale_path={scale_path.get()}\n")
        f.write(f"joid_data={joid_data.get()}\n")
        f.write(f"onlyOne={onlyOne.get()}\n")
        f.write(f"sampleNum={sampleNum.get()}\n")

def load_settings():
    try:
        with open('settings.txt', 'r') as f:
            lines = f.readlines()
            settings = {}
            for line in lines:
                key, value = line.strip().split('=')
                settings[key] = value
            return settings.get('input'), settings.get('output'), settings.get('calc_circles'), settings.get('get_measure'), settings.get('measure_distance'), settings.get('useDefaultScale'), settings.get('scale_path'), settings.get('joid_data'), settings.get('onlyOne'), settings.get('sampleNum')
    except FileNotFoundError:
        return None, None, None, None, None, None, None, None, None, None, None, None

def callback2():
    # Acceder a las rutas ingresadas
    input_dir = input_path.get()
    output_dir = output_path.get()
    print(f"Ruta de lectura: {input_dir}")
    print(f"Ruta de guardado: {output_dir}")
    save_settings(input_dir, output_dir, measure_distance)
    Label(win, text="Settings saved!", font=('Century 20 bold')).pack(pady=4)

# Crear una instancia de la ventana Tkinter
win = Tk()

# Configurar la geometría de la ventana
win.geometry("1000x750")

# Aplicar el tema "dark"
#ttk.set_theme('dark')


# Funciones para abrir el explorador de archivos y seleccionar una ruta de directorio
def select_input_path():
    folder_selected = filedialog.askdirectory()
    input_path.set(folder_selected)
    input_label.config(text="Input image directory: " + folder_selected)

def select_output_path():
    folder_selected = filedialog.askdirectory()
    output_path.set(folder_selected)
    output_label.config(text="Output image directory: " + folder_selected)

# Crear los widgets de entrada
input_path = StringVar()
input_button = Button(win, text="Select image input directory:", command=select_input_path)
input_button.pack()

input_label = Label(win, text="")
input_label.pack()

output_path = StringVar()
output_button = Button(win, text="Select image output directory:", command=select_output_path)
output_button.pack()

output_label = Label(win, text="")
output_label.pack()


# Crear un botón
btn = ttk.Button(win, text="Save predefined settings", command=callback2)
btn.pack(ipadx=10, pady=10)

def confirm_delete():
    confirm_win = Toplevel(win)
    confirm_win.geometry("200x100")
    Label(confirm_win, text="Are you sure?").pack()
    Button(confirm_win, text="Yes", command=delete_settings).pack()
    Button(confirm_win, text="No", command=confirm_win.destroy).pack()

def delete_settings():
    try:
        os.remove('settings.txt')
        input_path.set("")
        output_path.set("")
        calc_circles.set(False)
        get_measure.set(False)
        measure_distance.set(0)
        input_label.config(text="")
        output_label.config(text="")
        scale_label.config(text="")
        useDefaultScale.set(False)
        joid_data.set(False)
        onlyOne.set(False)
        sampleNum.set(0)
    except FileNotFoundError:
        pass

delete_button = ttk.Button(win, text="Delete all predefined settings", command=confirm_delete)
delete_button.pack(pady=10)

def callback():
    # Acceder a las rutas ingresadas
    input_dir = input_path.get()
    output_dir = output_path.get()
    print(f"Ruta de lectura: {input_dir}")
    print(f"Ruta de guardado: {output_dir}")
    time_str = "time: " + datetime.datetime.now().strftime("%H:%M:%S")
    Label(win, text=time_str, font=('Century 20 bold')).pack(pady=4)
    Label(win, text="Calculating circles...", font=('Century 20 bold')).pack(pady=4)
    processImages(input_dir,output_dir,measure_distance,get_measure.get(),calc_circles.get(),useDefaultScale.get(),scale_path.get(),joid_data.get(),onlyOne.get(),sampleNum.get())

#tickbox for the user to select the option to calculate the circles
# Crear la casilla de verificación
calc_circles = BooleanVar()
check_button = ttk.Checkbutton(win, text="Calculate Circles", variable=calc_circles)
check_button.pack()



####################################
# Scale checkbox y scale image file selector
frameScale = ttk.Frame(win)
frameScale.pack()

useDefaultScale = BooleanVar()

check_button = ttk.Checkbutton(frameScale, text="Use default scale", variable=useDefaultScale)
check_button.pack(side=LEFT)

def select_scale_path():
    folder_selected = filedialog.askopenfilename()
    scale_path.set(folder_selected)
    scale_label.config(text="Scale image file: " + folder_selected)

scale_path = StringVar()
scale_button = ttk.Button(win, text="Select default scale image directory:", command=select_scale_path)
scale_button.pack()

scale_label = Label(win, text="")
scale_label.pack()

##########################################
frameScale = ttk.Frame(win)
frameScale.pack()

onlyOne = BooleanVar()

check_button = ttk.Checkbutton(frameScale, text="Apply to the specified sample", variable=onlyOne)
check_button.pack(side=LEFT)

#quantity of the real distance
sampleNum = IntVar()
sampleEntry = ttk.Entry(frameScale, textvariable=sampleNum)
sampleEntry.pack(side=LEFT)
##########################################
joid_data = BooleanVar()
check_button = ttk.Checkbutton(win, text="Join data", variable=joid_data)
check_button.pack()

####################################
#tickbox para medir la distancia real
# Crear la casilla de verificación
get_measure = BooleanVar()
get_measure.set(True)  # Establecer get_measure como verdadero por defecto

# Crea un marco para contener la casilla de verificación y las entradas
frame = ttk.Frame(win)
frame.pack()

check_button = ttk.Checkbutton(frame, text="Measure the real distance", variable=get_measure)
check_button.pack(side=LEFT)

#cantidad de la distancia real
measure_distance = IntVar()
quantity_entry = ttk.Entry(frame, textvariable=measure_distance)
quantity_entry.pack(side=LEFT)

def check_get_measure(*args):
    if get_measure.get():
        quantity_entry.pack(side=LEFT)
    else:
        quantity_entry.pack_forget()

# Añade un rastreador a get_measure
get_measure.trace_add('write', check_get_measure)

# Crea un botón
btn = ttk.Button(win, text="Press Enter", command=callback)
btn.pack(ipadx=10)

win.bind('<Return>', lambda event: callback())


# Carga las rutas desde el archivo de configuración al iniciar el programa
input_dir, output_dir, calc_circles_value, get_measure_value, measure_distance_value, useDefaultScale_value, scale_path_value, joid_data_value, onlyOne_value, sampleNum_value = load_settings()
if input_dir:
    print(input_dir)
    input_path.set(input_dir)
    input_label.config(text="Input image directory: " + input_path.get())
if output_dir:
    output_path.set(output_dir)
    output_label.config(text="Output image directory: " + output_path.get())
if calc_circles_value is not None:
    calc_circles.set(calc_circles_value == 'True')
if get_measure_value is not None:
    get_measure.set(get_measure_value == 'True')
if measure_distance_value is not None:
    measure_distance.set(int(measure_distance_value))
if useDefaultScale_value is not None:
    useDefaultScale.set(useDefaultScale_value == 'True')
if scale_path_value is not None:
    scale_path.set(scale_path_value)
    scale_label.config(text="Scale image directory: " + scale_path.get())
if joid_data_value is not None:
    joid_data.set(joid_data_value == 'True')
if onlyOne_value is not None:
    onlyOne.set(onlyOne_value == 'True')
if sampleNum_value is not None:
    sampleNum.set(int(sampleNum_value))

sv_ttk.set_theme("dark")
win.mainloop()

G:/tfm_nanociencia/newinput2706
